In [32]:
import pygame
import sys

pygame.init()

# 화면 크기 설정
width, height = 800, 600
screen = pygame.display.set_mode((width, height))
pygame.display.set_caption("라면 게임")

# 색상 및 폰트 설정
YELLOW = (255, 255, 0)
BLUE = (0, 0, 255)
RED = (255, 0, 0)
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)

font_path = "C:/Windows/Fonts/malgun.ttf"
font = pygame.font.Font(font_path, 40)
small_font = pygame.font.Font(font_path, 30)

# 이미지 설정
home_img = pygame.transform.scale(pygame.image.load("main.png"), (width, height))
ending_img = pygame.transform.scale(pygame.image.load("ending.png"), (width, height))
instructions_img = pygame.transform.scale(pygame.image.load("First.png"), (width, height))
background_img = pygame.transform.scale(pygame.image.load("background2.png"), (width, height))

kettle_img = pygame.transform.scale(pygame.image.load("kettle.png"), (200, 200))
pot_img = pygame.transform.scale(pygame.image.load("pot.png"), (200, 200))
egg_img = pygame.transform.scale(pygame.image.load("egg.png"), (100, 100))
ramen_img = pygame.transform.scale(pygame.image.load("ramen.png"), (100, 100))
pa_img = pygame.transform.scale(pygame.image.load("pa.png"), (100, 100))
supoo_img = pygame.transform.scale(pygame.image.load("supoo.png"), (100, 100))
supoo_1_img = pygame.transform.scale(pygame.image.load("supoo1.png"), (100, 100))
bin_pot_img = pygame.transform.scale(pygame.image.load("binpot.png"), (200, 200))
boil_pot_img = pygame.transform.scale(pygame.image.load("boil.png"), (200, 200))
pa_pa_img = pygame.transform.scale(pygame.image.load("papa.png"), (100, 100))
supooboil_img = pygame.transform.scale(pygame.image.load("supooboil.png"), (200, 200))
waterpot_img = pygame.transform.scale(pygame.image.load("waterpot.png"), (200, 200))
finishramen_img=pygame.transform.scale(pygame.image.load("finishramen.png"), (200, 200))
paramen_img=pygame.transform.scale(pygame.image.load("paramen.png"), (200, 200))
supooramen_img=pygame.transform.scale(pygame.image.load("supooramen.png"), (200, 200))
sungong_img=pygame.transform.scale(pygame.image.load("sungong.png"), (width, height))


score = 0

class Button:
    def __init__(self, x, y, width, height, text, action=None, color=(255, 232, 73), transparent=False):
        self.rect = pygame.Rect(x, y, width, height)
        self.text = text
        self.action = action
        self.color = color
        self.transparent = transparent

    def draw(self, screen):
        if not self.transparent:
            pygame.draw.rect(screen, self.color, self.rect)
        text_surface = font.render(self.text, True, BLACK)
        screen.blit(text_surface, (self.rect.centerx - text_surface.get_width() // 2,
                                   self.rect.centery - text_surface.get_height() // 2))

    def is_clicked(self, pos):
        return self.rect.collidepoint(pos)

class Block:
    def __init__(self, color=None, x=0, y=0, size=50, image=None):
        self.color = color
        self.rect = pygame.Rect(x, y, size, size)
        self.dragging = False
        self.image = image
        if image:
            self.rect.size = image.get_size()

    def draw(self, screen):
        if self.image:
            screen.blit(self.image, self.rect.topleft)
        else:
            pygame.draw.rect(screen, self.color, self.rect)

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN and self.rect.collidepoint(event.pos):
            self.dragging = True
            mouse_x, mouse_y = event.pos
            self.offset_x = self.rect.x - mouse_x
            self.offset_y = self.rect.y - mouse_y
        elif event.type == pygame.MOUSEBUTTONUP:
            self.dragging = False
        elif event.type == pygame.MOUSEMOTION and self.dragging:
            mouse_x, mouse_y = event.pos
            self.rect.x = mouse_x + self.offset_x
            self.rect.y = mouse_y + self.offset_y



def show_score():
    score_text = font.render(f"돈: {score}", True, BLACK)
    screen.blit(score_text, (640, 100))

def evaluate_ramen_by_image(pot_list):
    global score
    if any(pot.image == finishramen_img for pot in pot_list):
        score += 3000
    elif any(pot.image != pot_img for pot in pot_list):
        score += 1000
    return score



def ending_screen():
    restart_button = Button(550, 500, 200, 50, "다시하기")
    running = True
    while running:
        screen.blit(ending_img, (0, 0))
        restart_button.draw(screen)
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN and restart_button.is_clicked(event.pos):
                loading_screen()
                return
def show_success_screen():
    restart_button = Button(550, 500, 200, 50, "다시하기", lambda: loading_screen(True))
    running = True
    while running:
        screen.blit(sungong_img, (0, 0))
        restart_button.draw(screen)
        pygame.display.update()
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()
            if event.type == pygame.MOUSEBUTTONDOWN and restart_button.is_clicked(event.pos):
                restart_button.action()
                return


class Pot:
    def __init__(self, x, y):
        self.state = "empty"
        self.rect = pygame.Rect(x, y, 200, 200)
        self.image = pot_img
        self.last_water_time = None
        self.ingredients = []
        self.dragging = False  # 드래그 상태
        self.offset_x = 0
        self.offset_y = 0

    def add_water(self):
        if self.state == "empty":
            self.state = "water"
            self.image = waterpot_img
            self.last_water_time = pygame.time.get_ticks()
            self.ingredients.append("water")

    def add_soup(self):
        if self.state == "boiled" or self.state == "water":
            self.state = "soup"
            self.image = supooboil_img
            if "soup" not in self.ingredients:
                self.ingredients.append("soup")

    def add_ramen(self):
        if self.state == "soup":
            self.state = "ramen"
            self.image = supooramen_img
            if "ramen" not in self.ingredients:
                self.ingredients.append("ramen")

    def add_pa(self):
        if self.state == "ramen":
            self.state = "pa"
            self.image = paramen_img
            if "pa" not in self.ingredients:
                self.ingredients.append("pa")

    def add_egg(self):
        if self.state == "pa":
            self.state = "finish"
            self.image = finishramen_img
            if "egg" not in self.ingredients:
                self.ingredients.append("egg")


    def update_state(self):
        if self.state == "water" and self.last_water_time:
            elapsed = (pygame.time.get_ticks() - self.last_water_time) / 1000
            if elapsed >= 3:  # 5초 이상 끓이면 상태 변경
                self.state = "boiled"
                self.image = boil_pot_img

    def draw(self, screen):
        screen.blit(self.image, self.rect.topleft)

    def handle_event(self, event):
        if event.type == pygame.MOUSEBUTTONDOWN and self.rect.collidepoint(event.pos):
            self.dragging = True
            mouse_x, mouse_y = event.pos
            self.offset_x = self.rect.x - mouse_x
            self.offset_y = self.rect.y - mouse_y
        elif event.type == pygame.MOUSEBUTTONUP:
            self.dragging = False
        elif event.type == pygame.MOUSEMOTION and self.dragging:
            mouse_x, mouse_y = event.pos
            self.rect.x = mouse_x + self.offset_x
            self.rect.y = mouse_y + self.offset_y

def start_game():
    global score
    score = 0

    # 재료 초기 위치
    egg_init = (25, 10)
    ramen_init = (185, 10)
    pa_init = (25, 90)
    soup_init = (185, 90)
    kettle_init = (360, 370)

    # 냄비 초기 위치들
    pot_inits = [
        (50, 300),
        (250, 300),
        (10, 400),
        (180, 400)
    ]


    yellow_block = Block(image=egg_img, x=egg_init[0], y=egg_init[1])
    blue_block = Block(image=ramen_img, x=ramen_init[0], y=ramen_init[1])
    green_block = Block(image=pa_img, x=pa_init[0], y=pa_init[1])
    soup_block = Block(image=supoo_img, x=soup_init[0], y=soup_init[1])
    kettle = Block(image=kettle_img, x=kettle_init[0], y=kettle_init[1])


    pot_list = [Pot(x, y) for x, y in pot_inits]

    for pot in pot_list:
        pot.state = "empty"
        pot.image = pot_img
        pot.last_water_time = None
        pot.ingredients = []

    submit_button = Button(580, 400, 120, 120, "제출", color=(255, 180, 50), transparent=True)

    start_ticks = pygame.time.get_ticks()
    total_time = 90  # 90초
    correct_order = True
    game_over = False

    while not game_over:
        elapsed_time = (pygame.time.get_ticks() - start_ticks) / 1000
        remaining_time = max(0, total_time - elapsed_time)

        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            if event.type == pygame.MOUSEBUTTONDOWN:
                if submit_button.is_clicked(event.pos):
                    # 제출 버튼 눌렀을 때 평가
                    cooking_times = []
                    for pot in pot_list:
                        if pot.last_water_time:
                            cooking_times.append((pygame.time.get_ticks() - pot.last_water_time) / 1000)
                        else:
                            cooking_times.append(0)
                    # 모든 냄비 중 하나라도 올바른 순서로 완성된게 있으면 성공
                    correct_order = any(
                        pot.ingredients == ["water", "soup","ramen", "pa", "egg"]
                        for pot in pot_list
                    )
                    # 가장 오래 끓인 냄비 기준으로 점수 평가
                    max_cook_time = max(cooking_times) if cooking_times else 0
                    evaluate_ramen_by_image(pot_list)

                    # 재료 초기 위치로 리셋
                    yellow_block.rect.topleft = egg_init
                    blue_block.rect.topleft = ramen_init
                    green_block.rect.topleft = pa_init
                    soup_block.rect.topleft = soup_init
                    kettle.rect.topleft = kettle_init       
                    for pot, pos in zip(pot_list, pot_inits):
                        pot.rect.topleft = pos
                        pot.state = "empty"
                        pot.image = pot_img
                        pot.last_water_time = None
                        pot.ingredients = []

            # 재료 및 주전자 드래그 처리
            yellow_block.handle_event(event)
            blue_block.handle_event(event)
            green_block.handle_event(event)
            soup_block.handle_event(event)
            kettle.handle_event(event)

            for pot in pot_list:
                pot.handle_event(event)

            # 냄비에 드롭시 상태 변화 처리
            if event.type == pygame.MOUSEBUTTONUP:
                for pot in pot_list:
                    if kettle.rect.colliderect(pot.rect):
                        pot.add_water()
                    if soup_block.rect.colliderect(pot.rect):
                        pot.add_soup()
                    if green_block.rect.colliderect(pot.rect):  # 파 블록
                        pot.add_pa()
                    if yellow_block.rect.colliderect(pot.rect):  # 계란 블록
                        pot.add_egg()
                    if blue_block.rect.colliderect(pot.rect):
                        pot.add_ramen()


        # 냄비 상태 업데이트
        for pot in pot_list:
            pot.update_state()

        # 배경 그리기
        screen.blit(background_img, (0, 0))
        # 냄비 그리기
        for pot in pot_list:
            pot.draw(screen)
        # 재료 및 주전자 그리기
        yellow_block.draw(screen)
        blue_block.draw(screen)
        green_block.draw(screen)
        soup_block.draw(screen)
        kettle.draw(screen)
        # 제출 버튼 그리기
        submit_button.draw(screen)
        show_score()

        if score >= 10000:
           show_success_screen()
           return

        # 남은 시간 바 그리기
        bar_width = 230
        bar_height = 30
        time_ratio = remaining_time / total_time
        pygame.draw.rect(screen, GRAY, (355, 85, bar_width, bar_height))
        pygame.draw.rect(screen, (233, 55, 113), (355, 85, int(bar_width * time_ratio), bar_height))

        if remaining_time <= 0:
            game_over = True

        pygame.display.update()

    pygame.time.wait(1000)
    ending_screen()

def loading_screen(show_instructions=False):
    start_button = Button(300, 460, 200, 50, "Start", start_game)
    instructions_button = Button(300, 540, 200, 50, "게임 설명")
    # 게임하기 버튼 (작게, 오른쪽 아래 등 원하는 위치/크기)
    play_button = Button(600, 500, 120, 50, "게임하기", start_game, color=(255, 232, 73))

    showing_instructions = False
    running = True

    while running:
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                pygame.quit()
                sys.exit()

            if event.type == pygame.MOUSEBUTTONDOWN:
                if not showing_instructions:
                    if start_button.is_clicked(event.pos):
                        if start_button.action:
                            start_button.action()
                    elif instructions_button.is_clicked(event.pos):
                        showing_instructions = True
                else:
                    if play_button.is_clicked(event.pos):
                        if play_button.action:
                            play_button.action()

        if showing_instructions:
            screen.blit(instructions_img, (0, 0))
            play_button.draw(screen)  # 게임설명 모드에서만 "게임하기" 버튼 그림
        else:
            screen.blit(home_img, (0, 0))
            start_button.draw(screen)
            instructions_button.draw(screen)

        pygame.display.update()


loading_screen()

SystemExit: 